     |████████████████████████████████| 1.4MB 5.1MB/s 
     |████████████████████████████████| 686kB 48.1MB/s 
     |████████████████████████████████| 296kB 46.3MB/s 
     |████████████████████████████████| 143kB 48.9MB/s 


In [5]:
!mkdir dataset
from google.colab import files
src = list(files.upload().values())[0]
open('\dataset','wb').write(src)
!mv sample_mobile_data_1000.csv dataset/
!ls dataset

mkdir: cannot create directory ‘dataset’: File exists


Saving sample_mobile_data_1000.csv to sample_mobile_data_1000.csv
sample_mobile_data_1000.csv


sample_mobile_data_1000.csv


ls: cannot access 'dataset': No such file or directory


In [7]:
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130811 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [8]:
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

KeyboardInterrupt: ignored

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive
 
print 'Files in Drive:'
!ls drive/
 
# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/created.txt

ModuleNotFoundError: No module named 'google'

In [4]:
!pip install -q -U tensor2tensor
!pip install -q tensorflow matplotlib

In [8]:
!conda install -y -c conda-forge tensorflow 


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3new

  added / updated specs:
    - tensorflow


The following NEW packages will be INSTALLED:

  _tflow_select      pkgs/main/win-64::_tflow_select-2.3.0-mkl
  absl-py            conda-forge/win-64::absl-py-0.7.1-py37_0
  astor              conda-forge/noarch::astor-0.7.1-py_0
  gast               conda-forge/noarch::gast-0.2.2-py_0
  grpcio             pkgs/main/win-64::grpcio-1.16.1-py37h351948d_1
  keras-applications conda-forge/noarch::keras-applications-1.0.7-py_1
  keras-preprocessi~ conda-forge/noarch::keras-preprocessing-1.0.9-py_1
  libmklml           pkgs/main/win-64::libmklml-2019.0.3-0
  libprotobuf        conda-forge/win-64::libprotobuf-3.7.1-h1a1b453_0
  markdown           conda-forge/noarch::markdown-2.6.11-py_0
  protobuf           conda-forge/win-64::protobuf-3.7.1-py37he025d50_0
  tensorboard        conda-forge/win-64::tensorboard-1.13.1-py37_0
  tensorflow   



==> WARNING: A newer version of conda exists. <==
  current version: 4.6.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base -c defaults conda



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3new




In [ ]:
!pip install tensorrec

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
checkpoint_dir = os.path.expanduser("~/t2t/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)
gs_data_dir = "gs://tensor2tensor-data"
gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [13]:
# A Problem is a dataset together with some fixed pre-processing.
# It could be a translation dataset with a specific tokenization,
# or an image dataset with a specific resolution.
#
# There are many problems available in Tensor2Tensor
problems.available()
# Fetch the MNIST problem
mnist_problem = problems.problem("image_mnist")
# The generate_data method of a problem will download data and process it into
# a standard format ready for training and evaluation.
mnist_problem.generate_data(data_dir, tmp_dir)
# Now let's see the training MNIST data as Tensors.
mnist_example = tfe.Iterator(mnist_problem.dataset(Modes.TRAIN, data_dir)).next()
image = mnist_example["inputs"]
label = mnist_example["targets"]

INFO:tensorflow:Not downloading, file already found: /root/t2t/tmp/train-images-idx3-ubyte.gz
INFO:tensorflow:Not downloading, file already found: /root/t2t/tmp/train-labels-idx1-ubyte.gz
INFO:tensorflow:Not downloading, file already found: /root/t2t/tmp/t10k-images-idx3-ubyte.gz
INFO:tensorflow:Not downloading, file already found: /root/t2t/tmp/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Not downloading, file already found: /root/t2t/tmp/train-images-idx3-ubyte.gz
INFO:tensorflow:Not downloading, file already found: /root/t2t/tmp/train-labels-idx1-ubyte.gz
INFO:tensorflow:Not downloading, file already found: /root/t2t/tmp/t10k-images-idx3-ubyte.gz
INFO:tensorflow:Not downloading, file already found: /root/t2t/tmp/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Skipping generator because outputs files exists at ['/root/t2t/data/image_mnist-unshuffled-train-00000-of-00010', '/root/t2t/data/image_mnist-unshuffled-train-00001-of-00010', '/root/t2t/data/image_mnist-unshuffled-train-00002-of-00010'

In [17]:
plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap('gray'))
print("Label: %d" % label.numpy())


Digit recognition :)



In [20]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("model_data/", one_hot=True)
import tensorflow as tf
batch =100
learning_rate=0.01
training_epochs=10
x = tf.placeholder(tf.float32, shape=[None, 784]) 
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting model_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting model_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting model_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting model_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

RuntimeError: ignored

In [0]:
y = tf.nn.softmax(tf.matmul(x,W) + b)